In [ ]:

import tensorflow as tf
import os
import pathlib
import matplotlib.pyplot as plt
import numpy as np

from tensorflow import keras
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Model

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Check for TensorFlow GPU access
print(tf.config.list_physical_devices())

# See TensorFlow version
print(tf.__version__)

In [ ]:
base_dir = '../data/fruits-360/'
test_dir = os.path.join(base_dir, 'Test')
train_dir = os.path.join(base_dir, 'Training')

train_dir = pathlib.Path(train_dir)
test_dir = pathlib.Path(test_dir) 

In [ ]:
# Img size

img_height = 224
img_width = 224

In [ ]:
# VGG16 Model 

# Data preprocessing - ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,# Image augmentaion 
    shear_range=0.2,
    zoom_range=0.2, # Zoom in on image by 20%
    horizontal_flip=True, #  Flip the image horizontally
    validation_split=0.2) # Split 20% of the data for validation

# Normalize the test data images
test_datagen = ImageDataGenerator(rescale=1./255) 

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='training') # Set as training data

validation_generator = train_datagen.flow_from_directory(
    train_dir, # Same directory as training data
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='validation') # Set as validation data

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical')

In [ ]:
# Create the model based on VGG16

base_model = tf.keras.applications.VGG16(include_top=False,
                                    weights="imagenet",
                                    input_shape=(img_height, img_width, 3))

Vgg = base_model.output
Vgg = GlobalAveragePooling2D()(Vgg)
#Vgg = Dropout(0.20)(Vgg)
predictions = Dense(131, activation= 'softmax')(Vgg)
Vgg = Model(inputs = base_model.input, outputs = predictions)

opt = Adam(lr=0.001)
Vgg.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

Vgg.summary()



In [ ]:
# Check for GPU availability
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [7]:
# Checkpoint train data
checkpoint = ModelCheckpoint("vgg16_1.h5", 
                             monitor='val_acc', verbose=1, save_best_only=True, 
                             save_weights_only=False, mode='auto', save_freq=1)

early = EarlyStopping(monitor='val_acc', 
                      min_delta=0, patience=20, verbose=1, mode='auto')

hist = Vgg.fit(x=train_generator, epochs=10, callbacks=[checkpoint,early], steps_per_epoch=100, 
                 validation_data=validation_generator,
                 validation_steps=10)



Epoch 1/10


2022-11-29 16:14:08.908874: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-29 16:14:09.474865: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


  6/100 [>.............................] - ETA: 1:27 - loss: 7.3970 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
# Save the model VGG
def save_model(model, seed):
    model.save(f"../models/vgg16v1-{seed}")
    
save_model(Vgg, 1)

In [ ]:
# Test the model

loss, accuracy = Vgg.evaluate(test_generator)
print(f'Test accuracy: {accuracy:.3f}, Test loss: {loss:.3f}')

In [ ]:
plt.plot(hist.history["acc"])
plt.plot(hist.history['val_acc'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()